# RPi Weather Station 
By Marcelo Rovai - June 2019

### List of Material 

<ol>
<li><a href="http://a.co/cSjzJhj"> Raspberry Pi V3</a> </li><li><a href="https://www.adafruit.com/product/385"> DHT22 Temperature and Relative Humidity Sensor</a></li><li>Resistor 4K7 ohm</li><li><a href="http://a.co/8NPkPQM"> DS18B20 Waterproof Temperature Sensor</a></li><li>Resistor 4K7 ohm</li><li><a href="http://a.co/3bhais0"> BMP180 Barometric Pressure, Temperature and Altitude Sensor </a></li><li><a href="https://es.aliexpress.com/item/UV-Detection-Sensor-Module-Ultraviolet-Ray-Module/32252554436.html?spm=a2g0s.9042311.0.0.jbbTqb" target="_blank">UV Sensor</a></li><li><a href="https://www.amazon.com/Adafruit-MCP3008-8-Channel-Interface-Raspberry/dp/B00NAY3RB2" target="_blank">Adafruit MCP3008 8-Channel 10-Bit ADC With SPI Interface</a></li></ol>

## DHT22 - Temperature & Humidity Sensor

In [1]:
# Adafruit DHT library (Temperature/Humidity)
import Adafruit_DHT

In [2]:
DHT22Sensor = Adafruit_DHT.DHT22
DHTpin = 16

In [3]:
humDHT, tempDHT = Adafruit_DHT.read_retry(DHT22Sensor, DHTpin)
if (humDHT is not None) and (tempDHT is not None):
    hum = round (humDHT,1)
    temp = round (tempDHT, 1)

In [4]:
print('Temperature = {}*C  Humidity = {}%'.format(temp, hum))

Temperature = 27.2*C  Humidity = 25.9%


---

## DS18B20 - Temperature Sensor

In [5]:
# DS18B20 1-Wire library (by default GPIO 4 is used with sensor (Temperature)
from w1thermsensor import W1ThermSensor 

In [6]:
ds18b20Sensor = W1ThermSensor() 

In [7]:
tempExt = round(ds18b20Sensor.get_temperature(), 1)

In [8]:
print('External Temperature = {}*C'.format(tempExt))

External Temperature = 35.6*C


---

## BMP180 - Temperature & Pressure Sensor

In [9]:
# BMP library (Pressure / Temperature)
import Adafruit_BMP.BMP085 as BMP085 

In [10]:
bmp180Sensor = BMP085.BMP085()

In [11]:
tempBMP = round(bmp180Sensor.read_temperature(), 1)
presBMP = round(bmp180Sensor.read_pressure()/100, 1) # absolute pressure in hPa
altBMP =  round(bmp180Sensor.read_altitude(),1)

In [12]:
print ('Temperature = {} C '.format(tempBMP))
print ('Pressure = {} hPa [or mbar]'.format(presBMP))
print ('Altitud = {} m '.format(altBMP))

Temperature = 27.7 C 
Pressure = 914.1 hPa [or mbar]
Altitud = 860.0 m 


### Defining a more complete function 

In [13]:
# Read data from BMP180
def bmp180GetData(altitude):

    temp = bmp180Sensor.read_temperature()
    pres = bmp180Sensor.read_pressure()
    alt =  bmp180Sensor.read_altitude()
    presSeaLevel = pres / pow(1.0 - altitude/44330.0, 5.255) 

    temp = round (temp, 1)
    pres = round (pres/100, 2) # absolute pressure in hPa (or mbar)
    alt = round (alt)
    presSeaLevel = round (presSeaLevel/100, 2) # absolute pressure in hPa (or mbar)

    return temp, pres, alt, presSeaLevel

In [14]:
altReal = 957 # station real altitud
tempLab, presAbs, altLab, presSL = bmp180GetData(altReal) 

In [15]:
print ("Station Air Temperature:       ", tempLab, "oC")
print ("Sea Level Air Pressure:        ", presSL, "mBar")
print ("Absolute Station Air Pressure: ", presAbs, "mBar")
print ("Station Calculated Altitude:   ", altLab, "m")

Station Air Temperature:        27.7 oC
Sea Level Air Pressure:         1025.33 mBar
Absolute Station Air Pressure:  914.23 mBar
Station Calculated Altitude:    859 m


---

## UV Sensor

In [45]:
import time
import spidev

In [17]:
# Open SPI bus
spi = spidev.SpiDev()
spi.open(0,0)
spi.max_speed_hz=1000000

In [18]:
# Function to read SPI data from MCP3008 chip
# Channel must be an integer 0-7
def ReadChannel(channel):
  adc = spi.xfer2([1,(8+channel)<<4,0])
  data = ((adc[1]&3) << 8) + adc[2]
  return data

In [19]:
UV_raw = ReadChannel(0)

In [20]:
print("UV Raw Analog Data: ", UV_raw)

UV Raw Analog Data:  128


### Calculating UV index

In [43]:
# Read UV sensor 3 times, take the average and converted it to mV
def readSensorUV():                   
    numOfReadings = 3
    dataSensorUV = 0
    for i in range(numOfReadings): 
        dataSensorUV += ReadChannel(0)
        time.sleep(0.2)
    dataSensorUV /= numOfReadings
    dataSensorUV = (dataSensorUV * (3.3 / 1023.0))*1000;
    return round(dataSensorUV)

In [44]:
def indexCalculate(dataSensorUV):
    if dataSensorUV < 227: indexUV = 0
    elif (227 <= dataSensorUV) & (dataSensorUV < 318): indexUV = 1
    elif (318 <= dataSensorUV) & (dataSensorUV < 408): indexUV = 2
    elif (408 <= dataSensorUV) & (dataSensorUV < 503): indexUV = 3
    elif (503 <= dataSensorUV) & (dataSensorUV < 606): indexUV = 4    
    elif (606 <= dataSensorUV) & (dataSensorUV < 696): indexUV = 5
    elif (696 <= dataSensorUV) & (dataSensorUV < 795): indexUV = 6
    elif (795 <= dataSensorUV) & (dataSensorUV < 881): indexUV = 7 
    elif (881 <= dataSensorUV) & (dataSensorUV < 976): indexUV = 8
    elif (976 <= dataSensorUV) & (dataSensorUV < 1079): indexUV = 9 
    elif (1079 <= dataSensorUV) & (dataSensorUV < 1170): indexUV = 10
    else: indexUV = 11  
    return indexUV

In [47]:
UV_raw = ReadChannel(0)
UV_mv = readSensorUV()
UV_index = indexCalculate(UV_mv)

print("UV raw data of {}, can be converted to {}mV, equivalent to UV-Index:\
{}.".format(UV_raw, UV_mv, UV_index ))

UV raw data of 7, can be converted to 20mV, equivalent to UV-Index:0.


---

## Capturing all adata

In [48]:
# time library 
import datetime

# Get data (from local sensors)
def getSensorData():
    global timeString
    global humLab
    global tempExt
    global tempLab
    global presSL
    global altLab
    global presAbs
    global UV_mV
    global UV_index

    # Get time of reading
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M")

    # Read External Temperature (1 meter distance)
    tempExt = round(ds18b20Sensor.get_temperature(), 1)

    tempLab, presAbs, altLab, presSL = bmp180GetData(altReal) 

    humDHT, tempDHT = Adafruit_DHT.read_retry(DHT22Sensor, DHTpin)
    if humDHT is not None and tempDHT is not None:
        humLab = round (humDHT)
        
    UV_mV = readSensorUV()
    UV_index = indexCalculate(UV_mV)   

In [53]:
# Display on screen important data
def printData():
    print ("Local Station Time:            ", timeString)
    print ("External Temperature (Aux.):   ", tempExt, "oC")
    print ("Station Air Temperature:       ", tempLab, "oC")
    print ("Station Air Humidity:          ", humLab, "%")
    print ("Sea Level Air Pressure:        ", presSL, "mBar")
    print ("Absolute Station Air Pressure: ", presAbs, "mBar")
    print ("Station Calculated Altitude:   ", altLab, "m")
    print ("UV:                            ", UV_mV, "mV")
    print ("Index UV:                      ", UV_index)

In [54]:
getSensorData()
printData()

Local Station Time:             2019-06-03 18:36
External Temperature (Aux.):    16.8 oC
Station Air Temperature:        21.5 oC
Station Air Humidity:           22 %
Sea Level Air Pressure:         1020.9 mBar
Absolute Station Air Pressure:  910.28 mBar
Station Calculated Altitude:    895 m
UV:                             0 mV
Index UV:                       0


---

## Logging Data

In [ ]:
with open("/home/pi/rpi_weather_station.csv", "a") as log:
    while True:
        getSensorData()
        log.write("{},{},{},{},{},{},{},{},{}\n".format(timeString, humLab, tempExt, tempLab, presSL, altLab, presAbs, UV_mV, UV_index))
        # print("{},{},{},{},{},{},{},{},{}".format(timeString, humLab, tempExt, tempLab, presSL, altLab, presAbs, UV_mV, UV_index))
        
        time.sleep(30)

In [68]:
def save_log():
    with open("/home/pi/rpi_weather_station.csv", "a") as log:
        log.write("{},{},{},{},{},{},{},{},{}\n".format(timeString, humLab, tempExt, tempLab, presSL, altLab, presAbs, UV_mV, UV_index))
    log.close()        

---

## Sending Data to ThingSpeak

In [ ]:
# Need to install the Paho client library.  
# sudo pip install paho-mqtt

In [60]:
import paho.mqtt.publish as publish
import psutil

ThingSpeak credentials and inicialization

In [61]:
channelID = "YOUR CHANNEL ID"
apiKey = "YOUR WRITE KEY"
topic = "channels/" + channelID + "/publish/" + apiKey
mqttHost = "mqtt.thingspeak.com"

MQTT Protocol Initialization

In [62]:
# Conventional TCP socket on port 1883.  
# This connection method is the simplest and requires the least system resources.
tTransport = "tcp"
tPort = 1883
tTLS = None

Defining Payload

In [63]:
 tPayload = "field1=" + str(humLab)+ "&field2=" + str(tempExt)+ "&field3=" + str(tempLab)+ "&field4=" + str(presSL)+ "&field5=" + str(altLab) + "&field6=" + str(presAbs)+ "&field7=" + str(UV_mV) + "&field8=" + str(UV_index)

Testing sending data one time

In [66]:
print ("[INFO] Data prepared to be uploaded")
try:
    publish.single(topic, payload=tPayload, hostname=mqttHost, port=tPort, tls=tTLS, transport=tTransport)
    print ("[INFO] Data sent for 8 fields: ", humLab, tempExt, tempLab, presSL, altLab, presAbs, UV_mV, UV_index)
except:
    print ("[INFO] Failure in sending data")

[INFO] Data prepared to be uploaded
[INFO] Data sent for 8 fields:  28 13.1 19.6 1021.55 890 910.86 0 0


Sending data continuosly

In [ ]:
# Sending all data to ThingSpeak every 1 minute
import time
while(True): 
    getSensorData()
    tPayload = "field1=" + str(humLab)+ "&field2=" + str(tempExt)+ "&field3=" + str(tempLab)+ "&field4=" + str(presSL)+ "&field5=" + str(altLab) + "&field6=" + str(presAbs)+ "&field7=" + str(UV_mV) + "&field8=" + str(UV_index)
    try:
        publish.single(topic, payload=tPayload, hostname=mqttHost, port=tPort, tls=tTLS, transport=tTransport)
        save_log()
    except (KeyboardInterrupt):
        break
    except:
        print ("[INFO] Failure in sending data") 
    time.sleep(60)